### Exploratoty DataAnalysis EDA

In [0]:
import pyspark.sql.functions as F

path_file = ("/Volumes/workspace/dataanalysispysparkbook/bronze_files/eda/")

my_grocery_list = [
    ["Banana", 2, 1.74],
    ["Apple", 4, 2.04],
    ["Carrot", 1, 1.09],
    ["Cake", 1, 10.99],
]

In [0]:
df_grocery_list = spark.createDataFrame(my_grocery_list, ["item", "quantity", "price"])
df_grocery_list.printSchema()
display(df_grocery_list)
df_grocery_list.coalesce(1).write.format("csv").mode("overwrite").save(path_file + "grocery_list.csv")

La estrategía para graficar información en PySpark es pasar una muestra del data frame a pandas con el método **toPandas()** y multiplicar las filas por las columnas, si el número es mayor a 100.000 (para un procesador de 16 GB) se debe reducir la muestra.

In [0]:
logs = spark.read.csv(
      path_file + "BroadcastLogs_2018_Q3_M8_sample.CSV", 
      sep = "|",
      header=True, 
      inferSchema = True,
      timestampFormat = "yyyy-MM.dd"
    )
display(logs.limit(5))

Exercise 4.1

In [0]:
sample = spark.read.csv(
    path_file + "grocery_list.csv", 
    sep = ",",
    header=True,
    quote = '"', 
    inferSchema = True   
)
display(sample)


In [0]:
logs.printSchema()

In [0]:
logs.select("BroadcastLogID", "LogServiceID", "LogDate").show(5, False)

El método select() es la otra forma de mantener solo las columnas que queremos

In [0]:
# Testing if we effectively got rid of the columns
print("BroadcastLogID" in logs.columns) # => True
print("SequenceNO" in logs.columns) # => True

logs = logs.select(*[c for c in logs.columns if c not in ("BroadcastLogID", "SequenceNO")])

# Testing if we effectively got rid of the columns
print("BroadcastLogID" in logs.columns) # => False
print("SequenceNo" in logs.columns) # => False

Drop columns


In [0]:
logs = logs.drop("BroadcastLogID", "SequenceNO")

# Testing if we effectively got rid of the columns
print("BroadcastLogID" in logs.columns) # => False
print("SequenceNo" in logs.columns) # => False

In [0]:
logs.select(F.col("Duration")).show(5)

In [0]:
logs.select(
    F.col("Duration"),
    F.col("Duration").substr(1, 2).cast("int").alias("dur_hours"),
    F.col("Duration").substr(4, 2).cast("int").alias("dur_minutes"),
    F.col("Duration").substr(7, 2).cast("int").alias("dur_seconds"),
).distinct().show(5)

Usando el método select()

In [0]:
logs.select(
    F.col("Duration"),
        (
            F.col("Duration").substr(1, 2).cast("int") * 60 * 60
            + F.col("Duration").substr(4, 2).cast("int") * 60
            + F.col("Duration").substr(7, 2).cast("int")
        ).alias("Duration_seconds"),
).distinct().show(5)

Usando el método withColumn()

In [0]:
logs = logs.withColumn(
    "Duration_seconds",
    (
        F.col("Duration").substr(1, 2).cast("int") * 60 * 60
        + F.col("Duration").substr(4, 2).cast("int") * 60
        + F.col("Duration").substr(7, 2).cast("int")
    ),
)
display(logs.limit(5))

In [0]:
logs = logs.withColumnRenamed("Duration_seconds", "duration_seconds")
logs.printSchema()

In [0]:
logs.toDF(*[x.lower() for x in logs.columns]).printSchema( )

In [0]:
logs.select(sorted(logs.columns)).printSchema()

In [0]:
for i in logs.columns:
    logs.describe(i).show()

In [0]:
for i in logs.columns:
    logs.select(i).summary().show()